<h1 style="font-size: 64px">Final data processing for Anime dataset</h1>

<h2 style="font-size: 48px">Importing modules</h2>

In [1]:
import pandas as pd
import numpy as np
from laserembeddings import Laser
import string
import copy
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import warnings
#import scipy
#from threading import Thread
#from threading import current_thread
#from timeit import default_timer
#import matplotlib as mpl
#import matplotlib.pyplot as plt
#import re
#import seaborn as sns
#from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
#from sklearn.feature_extraction.text import TfidfVectorizer
#import collections as coll
#from sklearn.cluster import KMeans
#from textblob import TextBlob
#import missingno as msno
#import spacy
#nlp = spacy.load('en_core_web_sm')
#Lets add some style
#import ast

In [2]:
#sns.set_style("dark")
warnings.simplefilter(action='ignore')
stop = stopwords.words('english')

In [6]:
path_to_bpe_codes = "Laser_embd_down/93langs.fcodes"
path_to_bpe_vocab = "Laser_embd_down/93langs.fvocab"
path_to_encoder = "Laser_embd_down/bilstm.93langs.2018-12-26.pt"
laser = Laser(path_to_bpe_codes, path_to_bpe_vocab, path_to_encoder)

In [7]:
df = pd.read_json("Clean_anime_general.json")
desc = pd.read_json("Description.json")
char_desc = pd.read_json("Character Description.json")
reviews = pd.read_json("Reviews embed.json")
shiet = pd.read_json("Processsed text.json", typ='series')

In [5]:
def embed_sentence(sentence, print_text=None):
    return laser.embed_sentences(sentence, lang="en").reshape(1024)

In [6]:
def LASER_this_columns(embed=None):
    if embed:
        for column in ["Description", "Character Description", "Reviews"]:
            last = df.index.max() + 1
            progress_bar = 5
            print_str = "[....................]"
            vectors = []
            for index in df.index:
                if (round(((index + 1) / last * 100), 5) // progress_bar) == 1:
                    print_str = list(print_str)
                    print_str[progress_bar // 5] = "■"
                    print_str = "".join(print_str)
                    progress_bar += 5
                print(f"{column}: finished for {print_str} {round(((index + 1) / last * 100), 2)}%      [{index + 1}/{last}]           ", end="\r")
                buffer = []
                for item in df[column][index]:
                    buffer.append(embed_sentence(item))
                if len(buffer) > 1:
                    vectors.append(pd.DataFrame(np.array(buffer)).mean().values)
                elif len(buffer) == 1:
                    vectors.append(np.array(buffer))
                else:
                    vectors.append(np.NaN)
            pd.DataFrame(vectors).to_json(f"{column}.csv")
            print(f"\n{column}.json file successfully written!")
    else:
        print("Description: finished for [■■■■■■■■■■■■■■■■■■■■] 100%      [18736/18736]")
        print("Description.json file successfully written!")
        print("Character Description: finished for [■■■■■■■■■■■■■■■■■■■■] 100%      [18736/18736]")
        print("Character Description.json file successfully written!")
        print("Reviews: finished for [■■■■■■■■■■■■■■■■■■■■] 100%      [18736/18736]")
        print("Reviews.json file successfully written!")

In [7]:
LASER_this_columns(embed=False)

Description: finished for [■■■■■■■■■■■■■■■■■■■■] 100%      [18736/18736]
Description.json file successfully written!
Character Description: finished for [■■■■■■■■■■■■■■■■■■■■] 100%      [18736/18736]
Character Description.json file successfully written!
Reviews: finished for [■■■■■■■■■■■■■■■■■■■■] 100%      [18736/18736]
Reviews.json file successfully written!


In [8]:
reviews = reviews.fillna(0)

In [23]:
char_desc_general = char_desc.mean()
reviews_general = reviews.mean()
desc_general = desc.mean()

In [10]:
#shiet = df["Description"].str.join(" ") + df["Character Description"].str.join(" ") + df["Reviews"].str.join(" ")
#shiet = shiet.fillna("")
#shiet = shiet.apply(lambda x: ' '.join([word for word in x.split() if word not in stop])).str.replace('[{}]'.format(string.punctuation), '')

In [11]:
#shiet

0        Substitute Soul Reaper Ichigo Kurosaki spends ...
1        After horrific alchemy experiment goes wrong E...
2        The elite members Shuchiin Academys student co...
3        Eccentric scientist Rintarou Okabe neverending...
4        Gintoki Shinpachi Kagura return funloving brok...
                               ...                        
18731    Once upon time two rival kingdoms get along Ev...
18732    No synopsis information added title Help impro...
18733    Mini anime shorts included four Bluray release...
18734    mobile fantasy RPG using characters manga maga...
18735    A world established people routinely swing sli...
Length: 18736, dtype: object

In [12]:
#ps = PorterStemmer()
#for index, item in zip(shiet.index, shiet):
#    print(f"[{index + 1}/{len(shiet)}]", end="\r")
#    stemmed = ""
#    words = word_tokenize(item)
#    for w in words:
#        stemmed += f"{ps.stem(w)} "
#    shiet[index] = stemmed

In [16]:
shiet

0        substitut soul reaper ichigo kurosaki spend da...
1        after horrif alchemi experi goe wrong elric ho...
2        the elit member shuchiin academi student counc...
3        eccentr scientist rintar okab neverend thirst ...
4        gintoki shinpachi kagura return funlov broke m...
                               ...                        
18731    onc upon time two rival kingdom get along ever...
18732    no synopsi inform ad titl help improv databas ...
18733     mini anim short includ four bluray releas n n f 
18734    mobil fantasi rpg use charact manga magazin br...
18735    a world establish peopl routin swing slightli ...
Length: 18736, dtype: object

In [13]:
'Story about organisation of assassins, that the main character was put in unknowingly. That organisation was created solely for the purpose of changing current Empire!'

'Story about organisation of assassins, that the main character was put in unknowingly. That organisation was created solely for the purpose of changing current Empire!'

In [28]:
print("[STARTING]...                                                                                     ")
# 1: Char description, 2: Reviews, 3: Description
query = 'Naruto'
query_show = copy.copy(query)
query = embed_sentence(query)
data = pd.DataFrame([query], index=["string"]).T
data["item"] = np.NaN

weights = [0.0, 0.0, 1.0]
if input("\nUse automatic weights? (y/n): ") == "y":
    buffer_w = pd.Series(index=['char_desc', 'reviews', 'desc'])
    data["item"] = char_desc_general
    buffer_w['char_desc'] = data.corr()["item"]["string"]
    data["item"] = reviews_general
    buffer_w['reviews'] = data.corr()["item"]["string"]
    data["item"] = desc_general
    buffer_w['desc'] = data.corr()["item"]["string"]

    suma = buffer_w["char_desc"] + buffer_w["reviews"] + buffer_w["desc"]
    weights[0] = (buffer_w["char_desc"] / suma) * 100
    weights[1] = (buffer_w["reviews"] / suma) * 100
    weights[2] = (buffer_w["desc"] / suma) * 100

query_preprocess = word_tokenize(pd.Series(query_show).apply(lambda x: ' '.join([word for word in x.split() if word not in stop])).str.replace('[{}]'.format(string.punctuation), '')[0])
stemmed = []
for w in query_preprocess:
    stemmed.append(ps.stem(w))
query_preprocess = copy.copy(stemmed)

new_feature = pd.Series(0, index=df.index)
last = len(query_preprocess)
for current, word in enumerate(query_preprocess):
    print(f"Preprocessing query...   [{current + 1}/{last}]", end="\r")
    new_feature += (shiet.str.find(word) > -1).astype("int64")
new_feature /= new_feature.max()
new_feature /= 2
    
test = char_desc.iloc[reviews.dropna().index] * weights[0] + reviews.dropna() * weights[1] + desc.iloc[reviews.dropna().index] * weights[2]
    
buff = pd.Series(index=reviews.dropna().index)
last = len(reviews.dropna())
progress_bar = 5
print_str = "[....................]"
current = 1
print(f"Progress: {print_str}                                                                            ", end='\r')
for index, item in zip(test.index, 
                       test.values):
    if (round((current / last * 100), 2) // progress_bar) == 1:
        print_str = list(print_str)
        print_str[progress_bar // 5] = "■"
        print_str = "".join(print_str)
        progress_bar += 5
        print(f"Progress: {print_str}                                                                            ", end='\r')
    current += 1
    data["item"] = item
    buff[index] = data.corr()["item"]["string"]
buff /= 2
buff += new_feature
print(f"\nQUERY: {query_show}\n")
print("TITLES")
print(df["Title"][buff.nlargest(24).index], end="\n\n")
print("GENRES")
print(df["Genres"][buff.nlargest(24).index], end="\n\n")
print("THEMES")
print(df["Themes"][buff.nlargest(24).index], end="\n\n")
print("WEIGHTS")
print(f"Description: {weights[2]}\nCharacter Description: {weights[0]}\nReviews: {weights[1]}")
print("PERCENTS\nCONTEXTUAL | WORDS")
for i in buff.nlargest(24).index:
    print(f"{i}   |   {buff[i] - new_feature[i]}   |   {new_feature[i]}")
#print(buff.nlargest(10))

[STARTING]...                                                                                     

Use automatic weights? (y/n): y
Progress: [■■■■■■■■■■■■■■■■■■■■]                                                                            
QUERY: Naruto

TITLES
1509               Naruto: Shippuuden - Sunny Side Battle
10314                                        Keraku no Ou
9764                     Wang Gu Shenhua Zhi Tian Xuanzhe
11005    Kamiusagi Rope x Boruto: Naruto Next Generations
7177                                   Kuiba: Shiwanhuoji
16305                                           Aqua Kids
4322                                        Ling Jian Zun
7319                                       Qian Cong Shou
5445                                       Xing Hai Qishi
3504                 Happy Birthday: Inochi Kagayaku Toki
8219                                                Blade
10345                          Yonhyaku-nijuu Renpai Girl
3385                            Anyamal T

In [103]:
df.Title.where(df.Title.str.find("Akame") > -1).dropna()

1781          Akame ga Kill!
5355    Akame ga Kill! Recap
Name: Title, dtype: object

In [ ]:
1781

In [111]:
buff.sort_values(ascending=False)[:24]

6523     0.641885
90       0.638567
3002     0.630761
2930     0.623368
495      0.615842
1586     0.613831
3270     0.611246
1191     0.610932
377      0.609242
10265    0.607617
3364     0.607388
2478     0.604837
11475    0.604193
369      0.603213
1982     0.602931
6818     0.601344
4923     0.601221
427      0.600440
1242     0.596457
2339     0.595543
3952     0.593507
5032     0.592771
759      0.589935
1781     0.589545
dtype: float64

In [102]:
df["Title"][buff.nlargest(20).index]

6523                                      AkaKill! Gekijou
90                                          Road of Naruto
3002                                              07-Ghost
2930                                 Groove Adventure Rave
495      Detective Conan Movie 18: The Sniper from Anot...
1586                                      Fate/Grand Order
3270      Naruto SD: Rock Lee no Seishun Full-Power Ninden
1191                                Dragon Ball Kai (2014)
377                                        Detective Conan
10265                      Boruto: Naruto Next Generations
3364     Naruto: Shippuuden - Shippuu! "Konoha Gakuen" Den
2478                                 Zettai Karen Children
11475                             Super Dragon Ball Heroes
369        Detective Conan Movie 20: The Darkest Nightmare
1982                                     Dragon Ball Super
6818                                        Dragon Ball GT
4923                            Toaru Majutsu no Index I

In [49]:
df["Description"][3]

['Eccentric scientist Rintarou Okabe has a never-ending thirst for scientific exploration. Together with his ditzy but well-meaning friend Mayuri Shiina and his roommate Itaru Hashida, Rintarou founds the Future Gadget Laboratory in the hopes of creating technological innovations that baffle the human psyche. Despite claims of grandeur, the only notable "gadget" the trio have created is a microwave that has the mystifying power to turn bananas into green goo.  However, when Rintarou decides to attend neuroscientist Kurisu Makise\'s conference on time travel, he experiences a series of strange events that lead him to believe that there is more to the "Phone Microwave" gadget than meets the eye. Apparently able to send text messages into the past using the microwave, Rintarou dabbles further with the "time machine," attracting the ire and attention of the mysterious organization SERN.  Due to the novel discovery, Rintarou and his friends find themselves in an ever-present danger. As he w

In [120]:
buff[75]

0.6029326232868121

In [116]:
df["Title"].where(df["Title"].str.find("Note") > -1).dropna()

75                                              Death Note
1133                                   Death Note: Rewrite
1607     Lord El-Melloi II Sei no Jikenbo: Rail Zeppeli...
2138     Lord El-Melloi II Sei no Jikenbo: Rail Zeppeli...
2413                                        Robotics;Notes
2521           Boku no Hero Academia 2nd Season: Hero Note
2887                        Soushisouai Note The Animation
3525     Lord El-Melloi II Sei no Jikenbo: Rail Zeppeli...
5107                                           Hinako Note
8307                             Tantei Team KZ Jiken Note
8743                           Ayatsuri Haramase DreamNote
9328                                     Girlfriend (Note)
9478                            Grimms Notes The Animation
10957    Shiritsutantei: Toki Shouzou Trouble Note - Ha...
12233                                           Tesla Note
12708                     Miko Note Episode 0: Zenjitsudan
13205               Nissan Note x The World of Golden Eg